Example arithmetic using the lambda calculus.

In [1]:
import time
from IPython.display import display, HTML, Math, Markdown
from lambda_calc import *


load_common_aliases()

def display_def(t):
    display(Math(t.to_latex(not_expanded={t})))

A straightforward pure normal order reduction can be very slow and involve very large intermediate structures. There are a number of potential speedups such as applicative order evaluation, lazy evaluation, use of references, and caching. In our implementation we consider an applicative speedup called "Λ expressions". The Λ experession "`(Λ x. M ) N`" is similar to the expression  "`(λ x. M) N`", except it is implemented as the capture avoiding substitution `(Λ x. M) N = M[x := normal_form(N)]` instead of `λ(x. M) N = M[x := N]`. This, of course, depends on `N` having a normal form.

The above lets the programmer specify which arguments can be eagerly evaluated. 

For example, here is the usual definitio of multiplication.

In [2]:
MULT_normal = λ['m'](λ['n'](λ['f'](λ['x'](('m', ('n', 'f')), 'x'))))

display_def(MULT_normal)

<IPython.core.display.Math object>

In [3]:
(MULT_normal | N(5) | N(3)).nf()[0]

λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x'))))))))))))))))

And here is a version of it with some of the arguments Λ-tagged as "eager."

In [4]:
MULT_eager = Λ['m'](Λ['n'](λ['f'](λ['x'](('m', ('n', 'f')), 'x'))))

display_def(MULT_eager)

<IPython.core.display.Math object>

In [5]:
(MULT_eager | N(5) | N(3)).nf()[0]

λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x'))))))))))))))))

We didn't see much of a difference, but there is quite a speedup when arguments are re-used.

In [6]:
GCDstep = λ["g"](λ["m", "n"](
    ifthenelse(
        (EQ, "m", "n"),
        "m",
        ifthenelse(
            (LEQ, "m", "n"),
            v("g") | (SUB, "n", "m") | "m",
            v("g") | (SUB, "m", "n") | "n",
        )
    )
))

GCDstep

λ['g'](λ['m'](λ['n'](((Λ['p'](λ['a'](λ['b'](('p', 'a'), 'b'))), ((Λ['m'](Λ['n']((λ['p'](λ['q'](('p', 'q'), 'p')), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'm'), 'n')), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'n'), 'm'))), 'm'), 'n')), 'm'), (((Λ['p'](λ['a'](λ['b'](('p', 'a'), 'b'))), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'm'), 'n')), (('g', ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'n'

In [7]:
start_time = time.time()
res = (Y | GCDstep | N(42) | N(56)).nf()[0]
end_time = time.time()

res

λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x')))))))))))))))

In [8]:
f"caclulation took {end_time - start_time:.2f} seconds"

'caclulation took 6.39 seconds'

In [9]:
GCDeager = λ["g"](Λ["m", "n"](
    ifthenelse(
        (EQ, "m", "n"),
        "m",
        ifthenelse(
            (LEQ, "m", "n"),
            v("g") | (SUB, "n", "m") | "m",
            v("g") | (SUB, "m", "n") | "n",
        )
    )
))

GCDeager

λ['g'](Λ['m'](Λ['n'](((Λ['p'](λ['a'](λ['b'](('p', 'a'), 'b'))), ((Λ['m'](Λ['n']((λ['p'](λ['q'](('p', 'q'), 'p')), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'm'), 'n')), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'n'), 'm'))), 'm'), 'n')), 'm'), (((Λ['p'](λ['a'](λ['b'](('p', 'a'), 'b'))), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'm'), 'n')), (('g', ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'n'

In [10]:
start_time = time.time()
res = (Y | GCDeager | N(42) | N(56)).nf()[0]
end_time = time.time()

res

λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x')))))))))))))))

In [11]:
f"caclulation took {end_time - start_time:.2f} seconds"

'caclulation took 1.71 seconds'

Another speedup is cached calculation.

In [12]:
expr = GCD | N(42) | N(56)

expr

v(((λ['g'](λ['m'](λ['n']((((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'm'), 'n'), (('g', 'n'), 'm')), (('g', 'm'), 'n')))), (λ['f'](λ['x']('f', ('x', 'x')), λ['x']('f', ('x', 'x'))), λ['g'](λ['x'](λ['y'](((Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), 'y'), 'x'), (('g', 'y'), ((Λ['a'](Λ['b'](λ['p']('p', λ['x'](λ['y']('y'))), ((((λ['f'](λ['x']('f', ('x', 'x')), λ['x']('f', ('x', 'x'))), λ['g'](λ['q'](λ['a'](λ['b']((((Λ['a'](Λ['b'](λ['p'](λ['a'](λ['b'](('p', 'b'), 'a'))), ((Λ['m'](Λ['n'](Λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), ((Λ['m'](Λ['n'](('n', Λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u'))))), 'm')), 'm'), 'n'))), 'b'), 'a'))), 'a'), 'b'), ((λ['x'](λ['y'](λ['f'](('f', 'x'), 'y'))), 'q'), 'a')), ((('g', (Λ['n'](λ['f'](λ['x']('f',

In [13]:
start_time_cached = time.time()
result_cached, steps_cached = expr.nf(tc=TransitiveCache())
end_time_cached = time.time()

result_cached

λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x')))))))))))))))

In [14]:
assert result_cached == N(14)

In [15]:
{"steps": steps_cached, "time": f"{end_time_cached - start_time_cached:.2f} seconds"}

{'steps': 38, 'time': '0.11 seconds'}

Non-caching calculation

In [16]:
start_time = time.time()
result, steps = expr.nf()
end_time = time.time()

result

λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x')))))))))))))))

In [17]:
assert result == N(14)

In [18]:
{"steps": steps, "time": f"{end_time - start_time:.2f} seconds"}

{'steps': 123, 'time': '4.44 seconds'}